In [ ]:
import gurobipy as gp
import itertools
from gurobipy import GRB

In [ ]:
def tsp_solver(distances):
    num_cities = len(distances)

    # Crear el modelo
    model = gp.Model()

    # Crear las variables
    x = {}
    for i in range(num_cities):
        for j in range(num_cities):
            x[i, j] = model.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}')

    # Restricción de visita
    for i in range(num_cities):
        model.addConstr(gp.quicksum(x[i, j] for j in range(num_cities) if j != i) == 1, name=f'visit_{i}')

    for j in range(num_cities):
        model.addConstr(gp.quicksum(x[i, j] for i in range(num_cities) if i != j) == 1, name=f'visit_{j}')

    # Restricción de subtours
    for subset_size in range(2, num_cities):
        for subset in itertools.combinations(range(num_cities), subset_size):
            model.addConstr(gp.quicksum(x[i, j] for i in subset for j in range(num_cities) if j not in subset) >= 1,
                            name=f'subtour_{subset}')

    # Función objetivo
    obj = gp.quicksum(distances[i][j] * x[i, j] for i in range(num_cities) for j in range(num_cities) if j != i)
    model.setObjective(obj, GRB.MINIMIZE)

    # Resolver el modelo
    model.optimize()

    # Obtener la solución
    if model.status == GRB.OPTIMAL:
        route = []
        for i in range(num_cities):
            for j in range(num_cities):
                if x[i, j].x > 0.5:
                    route.append((i, j))
        return route
    else:
        return None

In [ ]:
distances = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]
route = tsp_solver(distances)
print("Ruta óptima:")
for i, j in route:
    print(f"De ciudad {i} a ciudad {j}")